# Harry and Matty - CMPINF0010 Final Project

## Introduction + Metric

While determining the criteria for the "best" neighborhood in Pittsburgh we came up with many ideas from overall saftey to best area to raise children however we eventually decided upon the best place to hide a body. Now this may sound quite far fetched, but one day you may find yourself in a situation that would require you to take a few notes from this presentation so stay tuned. In determing what neighborhood would be the best fitted to hide a body, we decided to watch a few mafia movies and we found that remote areas lacking high vehicle traffic, lacking security cameras, and lacking arrests. This became the metric for our research into finding the "best" neighborhood in Pittsburgh. To begin this research we decided upon a few data sets recording a few different things. The first data set we chose recorded the number of cars that passed through different intersections around Pittsburgh. The next data set we chose recorded the number of security camera in use signs around different locations in Pittsburgh. And the last data set we decided upon was arrest data which we paired with population data to find average arrests per capita in each neighborhood.

## Conclusions

##### Harrison: 
    
##### Matt: